In [1]:
pip install statsforecast lowess statsmodels autogluon.timeseries TSDetective

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.4 MB/s eta 0:00:00
  

In [3]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from TSDetective.method import Detective, MultiplyN, Flip, AddN, SubN, ResidualFlip


raw = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")
airline_series = raw['#Passengers'].values
dates = raw['Month'].values

#doing this for chronos
df = pd.DataFrame({
    "timestamp": dates,
    "item_id": "my_series",
    "target": airline_series
})

In [4]:
holdout_length = 12

# Define transformations
transformations = [
    # Flip()
    MultiplyN(-1)
    # AddN(n=100000),  # Add 10 to the series
    # SubN(n=10),  # Subtract 10 from the series
    # ResidualFlip(seasonal_period=12)  # Apply LOWESS-based flipping
]

# Initialize the TSDetective
detective = Detective(
    original_series=df['target'].values,
    transformations=transformations,
    holdout_length=holdout_length,  # Use the last year for holdout
    seasonality=12  # Monthly seasonality
)

# Generate transformed series and split into training and holdout
transformed_series_split = detective.generate_transformed_series()

# Compute AutoARIMA error ratios
autoarima_ratios = detective.compute_autoarima_error_ratios()
print("AutoARIMA Error Ratios:", autoarima_ratios)

AutoARIMA Error Ratios: [1.0]


In [5]:


# Example foundation model errors (replace with actual values)
og_train_df = pd.DataFrame({
    "timestamp": dates[:-holdout_length],
    "item_id": "my_series",
    "target": airline_series[:-holdout_length]
})

synthetic_train_df = pd.DataFrame({
    "timestamp": dates[:-holdout_length],
    "item_id": "my_series",
    "target": transformed_series_split[0]['train']
})
# a predictor for the original series
predictor = TimeSeriesPredictor(prediction_length=12).fit(
    og_train_df,  hyperparameters={
        "Chronos": {'model_path': 'bolt_small'}}
)
og_chronos_predictions = predictor.predict(og_train_df, model='Chronos[bolt_small]')

# a predictor for the synthetic series
predictor = TimeSeriesPredictor(prediction_length=12).fit(
    og_train_df,  hyperparameters={
        "Chronos": {'model_path': 'bolt_small'}}
)
synthetic_chronos_predictions = predictor.predict(synthetic_train_df, model='Chronos[bolt_small]')

Beginning AutoGluon training...
AutoGluon will save models to '/content/AutogluonModels/ag-20250119_223213'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       11.01 GB / 12.67 GB (86.9%)
Disk Space Avail:   193.86 GB / 225.83 GB (85.8%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'bolt_small'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 12,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

Inferred time series frequency: 'MS'
Provided train_data has 132 rows, 1 time series. Median tim

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	-0.0348       = Validation score (-WQL)
	16.50   s     = Training runtime
	6.43    s     = Validation (prediction) runtime
Not fitting ensemble as only 1 model was trained.
Training complete. Models trained: ['Chronos[bolt_small]']
Total runtime: 22.96 s
Best model: Chronos[bolt_small]
Best model score: -0.0348
Beginning AutoGluon training...
AutoGluon will save models to '/content/AutogluonModels/ag-20250119_223243'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       10.55 GB / 12.67 GB (83.2%)
Disk Space Avail:   193.64 GB / 225.83 GB (85.7%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': {'model_path': 'bolt_small'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_lengt

In [6]:
#Calculate MSE for foundation model predictions on OG series
og_foundation_predictions = og_chronos_predictions['mean'].values
og_foundation_holdout = airline_series[-holdout_length:]
foundation_errors_original = detective.calculate_nmse(og_foundation_predictions, og_foundation_holdout)
#Calculate MSE for foundation model predictions on synthetic series
synthetic_foundation_predictions = synthetic_chronos_predictions['mean'].values
synthetic_foundation_holdout = transformed_series_split[0]["holdout"]
foundation_errors_transformed = detective.calculate_nmse(synthetic_foundation_predictions, synthetic_foundation_holdout)

In [7]:
# Compute foundation model error ratios
foundation_ratios = detective.compute_foundation_model_error_ratios(
    foundation_errors_original,
    foundation_errors_transformed
)
print("Foundation Model Error Ratios:", foundation_ratios)
print("AutoARIMA Model Error Ratios:", autoarima_ratios)
# Compare AutoARIMA and foundation model error ratios
comparison_ratios = detective.compare_error_ratios(autoarima_ratios, foundation_ratios)
print("Comparison Ratios (AutoARIMA / Foundation):", comparison_ratios)

Foundation Model Error Ratios: [0.5627491153522068]
AutoARIMA Model Error Ratios: [1.0]
Comparison Ratios (AutoARIMA / Foundation): [1.776990798775635]
